In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=3
✅ 시뮬레이션 초기화 완료: S=3, Randomness='high', M1=640.00, M2=1805.00


### 모델 DER Aggregation 

In [2]:
only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
agg = gp.Model("agg")
# agg.Params.OutputFlag = 0
agg.Params.MIPGap = 0.00001  # MIP gap을 1%로 설정


alpha = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
beta_plus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
beta_minus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")

M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
prob = np.array([1 / S for s in range(S)])

agg.update()

obj = gp.quicksum(P_DA[t] * alpha[t] for t in range(T)) + gp.quicksum(
    prob[s] * (P_RT[t, s] * beta_plus[t, s] - P_PN[t] * beta_minus[t, s])
    for t in range(T)
    for s in range(S)
)

agg.setObjective(obj, GRB.MAXIMIZE)

for t in range(T):
    for s in range(S):
        agg.addConstr(
            gp.quicksum(R[i, t, s] for i in range(I)) - alpha[t]
            == beta_plus[t, s] - beta_minus[t, s]
        )

for t in range(T):
    for s in range(S):
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= beta_plus[t, s])

for t in range(T):
    for s in range(S):
        agg.addConstr(beta_plus[t, s] <= M * z[t, s])
        agg.addConstr(beta_minus[t, s] <= M * (1 - z[t, s]))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20
Set parameter MIPGap to value 1e-05


### 결과 분석

#### 수익 분석

In [4]:
# Day-ahead 수익 계산
total_da_profit_obj = 0
for t in range(T):
    total_da_profit_obj += P_DA[t] * alpha[t].x  

# Real-time 수익 계산
total_rt_profit_obj = 0
for t in range(T):
    for s in range(S):
        rt_profit_obj = P_RT[t, s] * beta_plus[t, s].x  
        total_rt_profit_obj += prob[s] * rt_profit_obj

# Penalty 비용 계산
total_penalty_cost_obj = 0
for t in range(T):
    for s in range(S):
        penalty_cost_obj = P_PN[t] * beta_minus[t, s].x
        total_penalty_cost_obj += prob[s] * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익: {total_system_profit_obj:.2f}")

DA: 560779.62
RT: 1138504.90
Penalty: 61530.76
목적 함수 기반 총 이익: 1637753.76


#### 하루 aggregated 커밋량 분석

In [9]:
# 각 시간대 t에 대한 평균 alpha, beta+ 및 beta- 계산 및 출력
for t in range(T):
    avg_alpha = alpha[t].x  # alpha[t]의 최적화 값
    avg_beta_plus = sum(beta_plus[t, s].x for s in range(S)) / S  # beta_plus의 평균
    avg_beta_minus = sum(beta_minus[t, s].x for s in range(S)) / S  # beta_minus의 평균

    print(f"[시간 {t}] alpha: {avg_alpha:.1f}, beta+: {avg_beta_plus:.1f}, beta-: {avg_beta_minus:.1f}")

# alpha 값의 총합 출력
total_alpha = sum(alpha[t].x for t in range(T))
print(f"총 하루 commitment: {total_alpha:.1f}")

[시간 0] alpha: 0.0, beta+: 0.0, beta-: 0.0
[시간 1] alpha: 0.0, beta+: 0.0, beta-: 0.0
[시간 2] alpha: 0.0, beta+: 0.0, beta-: 0.0
[시간 3] alpha: 0.0, beta+: 0.0, beta-: 0.0
[시간 4] alpha: 0.0, beta+: 0.7, beta-: 0.0
[시간 5] alpha: 0.0, beta+: 0.0, beta-: 0.0
[시간 6] alpha: 3.0, beta+: 0.7, beta-: 0.7
[시간 7] alpha: 8.0, beta+: 2.7, beta-: 0.0
[시간 8] alpha: 58.0, beta+: 0.3, beta-: 12.0
[시간 9] alpha: 179.0, beta+: 16.3, beta-: 8.0
[시간 10] alpha: 432.0, beta+: 83.7, beta-: 0.0
[시간 11] alpha: 0.0, beta+: 543.3, beta-: 0.0
[시간 12] alpha: 0.0, beta+: 934.3, beta-: 0.0
[시간 13] alpha: 0.0, beta+: 1461.7, beta-: 0.0
[시간 14] alpha: 1446.0, beta+: 119.7, beta-: 243.7
[시간 15] alpha: 0.0, beta+: 989.0, beta-: 0.0
[시간 16] alpha: 640.0, beta+: 153.0, beta-: 0.0
[시간 17] alpha: 0.0, beta+: 775.7, beta-: 0.0
[시간 18] alpha: 563.0, beta+: 21.3, beta-: 0.0
[시간 19] alpha: 317.0, beta+: 145.7, beta-: 0.0
[시간 20] alpha: 65.0, beta+: 23.7, beta-: 0.0
[시간 21] alpha: 27.0, beta+: 1.0, beta-: 1.0
[시간 22] alpha: 0.0, beta

### 사후정산

In [10]:
only_value = pd.read_csv('result/result_only_obj.csv').values
only_profit = pd.read_csv('result/result_only_profit.csv').values
surplus = agg.objVal - only_value[0] #

I, T, S = R.shape

R_proportion = R / R.sum(axis=1, keepdims=True) 
R_proportion = np.nan_to_num(R_proportion) 

R_proportion_P = np.multiply(R_proportion, P_DA[:, np.newaxis])

R_weighted = R_proportion_P.sum(axis=(1, 2)) 

R_weighted_normalized = R_weighted / R_weighted.sum() 

surplus_distribution = surplus * R_weighted_normalized  

final_profit = only_profit.flatten() + surplus_distribution

print("Surplus 분배 결과:")
for i, value in enumerate(surplus_distribution):
    print(f"[{i}]: {value:.2f}")

print("\n최종 Profit:")
for i, (profit, only) in enumerate(zip(final_profit, only_profit.flatten())):
    increase_percentage = ((profit - only) / only) * 100
    print(f"[{i}] {profit:.2f} ({increase_percentage:.2f}%)")

Surplus 분배 결과:
[0]: 1413.00
[1]: 1430.17
[2]: 1438.11
[3]: 1485.18
[4]: 1460.34

최종 Profit:
[0] 223932.57 (0.64%)
[1] 332341.27 (0.43%)
[2] 369936.39 (0.39%)
[3] 544194.10 (0.27%)
[4] 167349.43 (0.88%)


### 결과 저장

In [11]:
alpha_df = pd.DataFrame({
    'alpha': [alpha[t].x for t in range(T)]
})
alpha_df.to_csv('result/result_base_alpha.csv', index=False)

# # beta_plus 저장
# beta_plus_df = pd.DataFrame(
#     [[beta_plus[t,s].x for s in range(S)] 
#      for t in range(T)],
#     columns=[f'S{s}' for s in range(S)]
# )
# beta_plus_df.to_csv('result_beta_plus.csv', index=False)

# # beta_minus 저장
# beta_minus_df = pd.DataFrame(
#     [[beta_minus[t,s].x for s in range(S)] 
#      for t in range(T)],
#     columns=[f'S{s}' for s in range(S)]
# )
# beta_minus_df.to_csv('result_beta_minus.csv', index=False)

In [12]:
agg_profit_value = pd.DataFrame({'agg_profit_value': final_profit})
agg_profit_value.to_csv('result/result_agg_profit.csv', index=False)

agg_obj = pd.DataFrame({'agg_obj': [agg.objVal]})
agg_obj.to_csv('result/result_agg_obj.csv', index=False)